# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
harvesting:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
producer2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
purchase:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [5]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

Explain your answer
Describe the schema of new tables and how they relate


| Attribute | Status | Description |
| --- | --- | --- |
| NP     | Key | Producer number |
| NOM    |  | Producer's last name  |
| PRENOM |  | Producer's first name |
| REGION |  | Producer region       |

| Attribute | Status | Description |
| --- | --- | --- |
| NV     | Key | Wine number |
| CRU     |  | Vineyard or group of vineyards|
| DEGREE     |  | Alcohol content |
| MILL     |  | Vintage year |

| Attribute | Status | Description |
| --- | --- | --- |
| NV     | Key | Wine number |
| NP     | Key | Producer number |
| QTE     |  | Number of bottles harvested |

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION 

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate


| Attribute | Status | Description |
| --- | --- | --- |
| NB     | Key | Producer number |
| NOM    |  | Producer's last name  |
| PRENOM |  | Producer's first name |
| TYPE |  | Type of client by volume of purchases       |

| Attribute | Status | Description |
| --- | --- | --- |
| NV     | Key | Wine number |
| CRU     |  | Vineyard or group of vineyards|
| DEGREE     |  | Alcohol content |
| MILL     |  | Vintage year |

| Attribute | Status | Description |
| --- | --- | --- |
| NV     | Key | Wine number |
| NB     | Key | Producer number |
| QTE     |  | Number of bottles harvested |
|DATES|  | Buying date |
|LIEU |KEY | Place where the wine was sold |


| Attribute | Status | Description |
| --- | --- | --- |
| LIEU     | Key | Place where the wine was sold |
| REGION     |  | Administrative Region (different to production region |

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [6]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
harvesting:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
producer2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
purchase:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)

Content of dummy
 * sqlite://

DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [7]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [8]:
%%sql

DROP TABLE IF EXISTS producer;

CREATE TABLE producer AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
%%sql

DROP TABLE IF EXISTS wine;

CREATE TABLE wine AS
SELECT DISTINCT NV, CRU,DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL

 * sqlite:///wine.db
Done.
Done.


[]

In [10]:
%%sql

DROP TABLE IF EXISTS harvesting;

CREATE TABLE harvesting AS
SELECT DISTINCT NP,NV,QTE
FROM MASTER1
WHERE NP IS NOT NULL
      OR NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

__1.4__ Create the new tables from Master2:

In [11]:
%%sql

DROP TABLE IF EXISTS client;

CREATE TABLE client AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [12]:
%%sql

DROP TABLE IF EXISTS wine2;

CREATE TABLE wine2 AS
SELECT DISTINCT NV, CRU,DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL

 * sqlite:///wine.db
Done.
Done.


[]

# Here we notice that creating a wine table for MASTER2 is no use. In fact it is the same table as for MASTER1

In [39]:
%%sql

DROP TABLE IF EXISTS purchase;

CREATE TABLE purchase AS
SELECT DISTINCT NB, NV, DATES, LIEU, QTE 
FROM MASTER2
WHERE NB IS NOT NULL
      AND NV IS NOT NULL
      AND LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [40]:

%%sql 

DROP TABLE IF EXISTS location;

CREATE TABLE location AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

In [41]:
%%sql

SELECT DISTINCT TYPE 
FROM client;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


__2.2__ What regions produce Pommard or Brouilly?

In [48]:
%%sql

SELECT DISTINCT REGION, CRU
FROM producer as p , wine as w, harvesting as h
WHERE w.CRU='Pommard' OR w.CRU='Brouilly'
      AND p.NP=h.NP
      AND h.NV=w.NV;

 * sqlite:///wine.db
Done.


REGION,CRU
Alsace,Pommard
Bourgogne,Pommard
Beaujolais,Pommard
Jura,Pommard
Rhone,Pommard
Provence,Pommard
Corse,Pommard
Languedoc,Pommard
Pyrenees,Pommard
Sud,Pommard


__2.3__ What regions produce Pommard and Brouilly?

In [52]:
%%sql

SELECT REGION
FROM producer as p , wine as w, harvesting as h
WHERE w.CRU='Pommard'
      AND p.NP=h.NP
      AND h.NV=w.NV
        
INTERSECT

SELECT REGION
FROM producer as p , wine as w, harvesting as h
WHERE w.CRU='Brouilly'
      AND p.NP=h.NP
      AND h.NV=w.NV;


 * sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bught by CRU and Millésime

In [54]:
%%sql

SELECT CRU, MILL, SUM(QTE) AS WINES_number

FROM wine, harvesting

WHERE wine.NV = harvesting.NV

GROUP BY CRU, MILL

ORDER BY CRU,MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,WINES_number
Arbois,1976,69
Arbois,1980,2175
Auxey Duresses,1914,1250
Beaujolais Primeur,1983,80
Beaujolais Villages,1975,800
Beaujolais Villages,1976,2110
Beaujolais Villages,1978,450
Beaujolais Villages,1979,100
Bellet,1976,45
Blanquette de Limoux,1978,110


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [56]:
%%sql

SELECT NV, COUNT(NP) AS N_PRODUCERS
FROM harvesting
WHERE NV IS NOT NULL
GROUP BY NV
HAVING COUNT(NP) > 3;


 * sqlite:///wine.db
Done.


NV,N_PRODUCERS
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

In [59]:
%%sql
SELECT p.NP, p.NOM, p.PRENOM
FROM wine as w, harvesting as h, producer as p
EXCEPT
SELECT p.NP, p.NOM, p.PRENOM
FROM wine as w, harvesting as h, producer as p
WHERE w.NV=h.NV and p.NP=h.NP
GROUP BY p.NP
HAVING count(h.QTE)>0
ORDER BY p.NP;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

In [61]:
%%sql 

SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client as c, purchase as p, wine as w
WHERE c.NB = p.NB
      AND w.MILL = '1980'
      AND w.NV = p.NV
ORDER BY c.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

In [62]:
%%sql 

SELECT DISTINCT NB, NOM, PRENOM
FROM client as c
EXCEPT
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client as c, purchase as p, wine as w
WHERE c.NB = p.NB
      AND p.NV = w.NV
      AND w.MILL = '1980'
ORDER BY c.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


__2.9__ What clients (buveurs) have bought ONLY wines from 1980?

In [63]:
%%sql

SELECT DISTINCT client.NB, client.NOM, client.PRENOM
FROM client, purchase, wine
WHERE wine.MILL = '1980'
      AND wine.NV = purchase.NV
      AND purchase.NB = client.NB
EXCEPT
SELECT DISTINCT client.NB, client.NOM, client.PRENOM
FROM client, purchase, wine
WHERE wine.MILL NOT IN ('1980')
      AND wine.NV = purchase.NV
      AND purchase.NB = client.NB
ORDER BY client.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


__2.10__ List all wines from 1980

In [64]:
%%sql 

SELECT *
FROM wine
WHERE MILL = '1980'
ORDER BY NV;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


__2.11__ What are the wines from 1980 bought by NB=2?

In [65]:
%%sql 

SELECT wine.NV, wine.CRU, wine.MILL, wine.DEGRE
FROM wine, purchase
WHERE wine.MILL = '1980'
      AND purchase.NB = '2'
      AND wine.NV = purchase.NV
ORDER BY wine.NV;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

In [66]:
%%sql 

SELECT client.NB, client.NOM, client.PRENOM
FROM client, purchase, wine 
WHERE purchase.NB = client.NB
      AND purchase.NV = wine.NV
GROUP BY client.NB
HAVING COUNT(wine.NV) = ( 
        SELECT COUNT(wine.NV)
        FROM wine
        WHERE wine.MILL = '1980');

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
